# Processing Word with XML

## Replace centent

In [13]:
import zipfile
templateDocx = zipfile.ZipFile("data/Template.docx")
newDocx = zipfile.ZipFile("data/NewDocument.docx", "a")


for file in templateDocx.filelist:
    templateDocx.extract(file.filename, "data")

with open("data/word/document.xml", encoding='utf-8') as docXmlFile:
    docXmlStr =  docXmlFile.read()
    # print(docXmlStr)
       
replaceText={'第五节':'这是替换的文本'} 
for key in replaceText.keys():
    new_docXmlStr = docXmlStr.replace(str(key), str(replaceText.get(key)))

with open("data/word/new_document.xml", "w+", encoding='utf-8') as new_docXmlFile:
    new_docXmlFile.write(new_docXmlStr)

for file in templateDocx.filelist:
    if not file.filename == "word/document.xml":
        newDocx.writestr(file.filename, templateDocx.read(file))

newDocx.write("data/word/new_document.xml", "word/document.xml")

templateDocx.close()
newDocx.close()

## Processing fontnote with footnotes.xml

In [14]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open("data/word/footnotes.xml", encoding='utf-8'), 'xml')
# print(soup.prettify())
for footnote in soup.find_all('w:footnote'):
    footnote_id = footnote.get('w:id')
    print( footnote_id,)
    print(footnote.text, '\n ','--'*10)


-1
 
  --------------------
0
 
  --------------------
1
 .《关于成立‘江苏东大特种基础工程开发公司’的报告》，1-2-1。 
  --------------------
2
 .《联营协议书》，1-2-1。 
  --------------------
3
 .《关于成立“江苏东大特种基础工程开发公司”的决定》，1-2-1。 
  --------------------
4
 .验字（94）6号验资证明，1-3-2。 
  --------------------
5
 .《国有资产产权登记表附表》，1-2-1。 
  --------------------
6
 .《审计报告》（XYZH/2015NJA30090号），1-4-4-1。 
  --------------------
7
 《评估报告》（天源评报字[2015]第0199号），1-4-4-2。 
  --------------------
8
 《验资报告》（XYZH[20l5]NJA30092号），1-4-4-3。 
  --------------------


In [15]:
for footnote in soup.find_all('w:footnote'):
    for w_t in footnote.find_all('w:t'):
        if  "关于成立" in w_t:
            print(w_t)
            w_t.insert(1,'测试')
            print(w_t)

with open('data/word/new_footnotes.xml', 'w',  encoding='utf-8') as fp:
    fp.write(soup.prettify())

<w:t>关于成立</w:t>
<w:t>关于成立测试</w:t>


# Processing Word with python-docx

## get paragraph text

In [1]:
from docx import Document

class Docx_extract:
    def __init__(self, fp):
        self.document = Document(fp)
        
    def get_paragraph(self):
        '''
        Parameters
        ----------
        f_p: docx file path

        Returns
        ----------
        out: generator
        Each element is A tuple containing (paragraph_number, paragraph.text)
        '''
        for  paragraph_number, paragraph in enumerate(self.document.paragraphs):
            # print(f'paragraph_number:{paragraph_number}, paragraph text:{paragraph.text}')
            paragraph_text = paragraph.text
            yield  paragraph_number, paragraph_text

if __name__=="__main__":
   
    fp = 'data/Template.docx'
    docx_extract = Docx_extract(fp)
    result= docx_extract.get_paragraph()
    print(list(result))


[(0, '第五节 发行人基本情况'), (1, '一、发行人基本情况'), (2, '二、发行人设立、报告期内股本变化、重大资产重组及在其他证券市场挂牌情况'), (3, '（一）有限公司成立情况'), (4, '发行人前身为江苏东大特种基础工程开发公司，由南京东大科技实业（集团）总公司和宜兴京宜特种地基基础工程公司共同出资设立。'), (5, '1993年11月25日，东大科技与宜兴京宜联合向江苏省工商行政管理局提交“关于成立‘江苏东大特种基础工程开发公司’的报告” ，申请设立“江苏东大特种基础工程开发公司”（即鸿基有限）。'), (6, '1993年12月2日，东大科技与宜兴京宜签署《联营协议书》，共同设立“江苏东大特种基础工程开发公司”。'), (7, '1993年12月6日，东大科技作出“东集发展字[93]第58号”决定，由东大科技所属基础与建材研究所与宜兴京宜联合成立“江苏东大特种基础工程开发公司”。'), (8, '1994年1月26日，江苏兴隆会计师事务所出具“验字（94）6号”《验资证明》，对宜兴京宜出资的价值324万元的机械设备进行验资。'), (9, '1994年2月4日，江苏省国有资产管理局出具《国有资产产权登记表附表》登记东大科技用于出资的固定资产为31万元，流动资金60万元。江苏省国有资产管理局对该附表所记载资产状况审核同意注册。'), (10, '1994年3月30日，鸿基有限完成工商注册手续并领取了江苏省工商行政管理局核发的《企业法人营业执照》（注册号13477645-3）。鸿基有限设立时的股权结构如下：'), (11, '（二）股份有限公司设立情况'), (12, '2015年7月15日，信永中和出具《审计报告》（XYZH/2015NJA30090号），根据该报告，有限公司截至2015年6月30日的净资产值为60,553,667.60元。'), (13, '2015年7月16日，天源资产评估有限公司出具《评估报告》（天源评报字[2015]第0199号），根据该报告，有限公司截至2015年6月30日的净资产的评估值为62,665,444.14元。'), (14, '2015年8月3日，有限公司召开股东会，全体股东一致同意，鸿基有限变更为股份有限公司，并以信永中和出具的《审计报告》（XYZH/20l5NJA30090号）确认的鸿基有限截

# Docx to byte stream and byte stream to docx 

In [5]:
import base64

fp = 'data/Template.docx'
with open(fp,'rb') as f:
    res = base64.b64encode(f.read())

docx = base64.b64decode(res)

with open('test.docx','wb') as f:
    f.write(docx)